So this one is for assignment 3 this week which is using deltaR between the event with n (and/or one with pi+) and the various clusters. Then minimize that to find the cluster that most likely contains the n (this is for delta+ I guess? Then do again for pi0 and p) 

Will be plotting the response plot (those predictions) of that one cluster specifically! Then compare these to the single pion sample.

This is the one that can't seem to work and I have nooooo idea why. I'll leave the messiness in this one for you to try to figure out:))

In [1]:
#imports
import uproot as ur
import awkward as ak
import numpy as np

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "6" #specify GPU
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

# energyflow imports
import energyflow as ef
from energyflow.archs import PFN
from energyflow.utils import data_split

from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

import graph_util as gu
import plot_util as pu

from scipy.interpolate import interp1d

from Week7_PFNutils import *

2021-07-23 08:00:47.763643: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-07-23 08:01:03.044476: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-07-23 08:01:03.077245: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-07-23 08:01:03.249859: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1746] Found device 0 with properties: 
pciBusID: 0000:3d:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2021-07-23 08:01:03.249960: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-07-23 08:01:03.319747: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2

In [2]:
model_path = '/fast_scratch/atlas_images/v01-45/' 
data_path = '/data/atlas/'

In [3]:
import vector as v

import atlas_mpl_style as ampl #makes matplotlib follow atlas formal style
ampl.use_atlas_style()

v.register_awkward()

In [4]:
events = ur.open(model_path+'delta_medium.root')

tree = events['EventTree']

In [5]:
branches = tree.arrays()

In [6]:
truthE = branches.cluster_hitsTruthE
truthIndex = branches.cluster_hitsTruthIndex
truthID = branches.truthPartPdgId
trutheta = branches.truthPartEta
truthphi = branches.truthPartPhi
truthpt = branches.truthPartPt
truthmass = branches.truthPartMass
clusE = branches.cluster_E
clusEta = branches.cluster_Eta
clusPhi = branches.cluster_Phi
clusPt = branches.cluster_Pt

In [7]:
#start with just the delta+ -> n + pi+
nnopind = []    

for i in range(len(truthID)): #basically: delta+, two categories: pip & n (without any ps) vs pi0 & p (no ns)
    if (2214 in truthID[i]) & ~(2212 in truthID[i]) & (2112 in truthID[i]) & (211 in truthID[i]):
        nnopind.append(i)
        
flag = np.zeros(len(truthID))
flag[nnopind] = 2

In [8]:
mask = flag == 2

In [9]:
truthE_sel = truthE[mask]
truthIndex_sel = truthIndex[mask]
truthID_sel = truthID[mask]
trutheta_sel = trutheta[mask]
truthphi_sel = truthphi[mask]
truthpt_sel = truthpt[mask]
truthmass_sel = truthmass[mask]
clusE_sel = clusE[mask]
clusEta_sel = clusEta[mask]
clusPhi_sel = clusPhi[mask]
clusPt_sel = clusPt[mask]

In [25]:
#truthIndex_sel[100], truthID_sel[100], trutheta_sel[100] #okay so don't need the index, just ID

(<Array [[1], [1]] type='2 * var * int64'>,
 <Array [2214, 2112, 211] type='3 * int32'>,
 <Array [-2.33, -2.18, -3.77] type='3 * float32'>)

In [37]:
#for i in range(len(truthID_sel)):
 #   if len(truthID_sel[i]) != len(trutheta_sel[i]):
  #      print('uh oh') #okay so this should work

In [28]:
clusvec = ak.zip({
    "pt": clusPt_sel[0],
    "eta": clusEta_sel[0],
    "phi": clusPhi_sel[0],
    "mass": clusmass_sel[0],
})

In [35]:
print(clusvec[1]) #so it flattens it?? frick

{pt: 0.217, eta: 1.79, phi: 1.3, mass: 0}


In [34]:
clusEta_sel[0]

<Array [-1.96, 1.79] type='2 * float32'>

In [10]:
clusmass_sel = []
for i in range(len(clusE_sel)):
    clusmass_sel.append(np.zeros(len(clusE_sel[i])))#okay great this is how to get it the right shape:))

In [19]:
clusEta_sel[0], trutheta_sel[0] #so there are 4 particles, two clusters.....
#for each event, using only the n, loop through the clusters and keep the closest one. okay

(<Array [-1.96, 1.79] type='2 * float32'>,
 <Array [-2.21, -1.81, -2.12, -2.13] type='4 * float32'>)

# I HAVE NO IDEA 

In [11]:
#this function is deleted in the other one cause it didn't seem to help. oh well.
def findmin(clusPt_sel_i, clusEta_sel_i, clusPhi_sel_i, clusmass_sel_i, part4Dvec):
    deltaR = [] #this should work..... I'm sure of it
    for j in range(len(clusEta_sel_i)):
        #print('nclusters ', len(clusEta_sel_i))
        clusvec = ak.zip({
            "pt": clusPt_sel_i[j],
            "eta": clusEta_sel_i[j],
            "phi": clusPhi_sel_i[j],
            "mass": clusmass_sel_i[j]})
        clus4Dvec = ak.with_name(clusvec, "Momentum4D")
        
        deltaR.append(part4Dvec.deltaR(clus4Dvec))
        #print('flat', np.array(ak.flatten(deltaR)))
        #print('normal', deltaR)
        print(np.array(ak.flatten(deltaR)))
        if deltaR == [1.3131634]:
            print('exception: empty deltaR') #okay so this does work I just needed to restart the kernel:))
            print(part4Dvec, clus4Dvec, part4Dvec.deltaR(clus4Dvec), deltaR, i, j)
            error = part4Dvec.deltaR(clus4Dvec)
            return error
    
    ind = np.argmin(np.array(ak.flatten(deltaR)))#np.where(deltaR == min(np.array(ak.flatten(deltaR))))# 
    
    return ind

In [12]:
mindist_indices = []
for i in range(len(trutheta_sel)): #loop through each event
    #select the neutron:
    n_mask = truthID_sel[i] == 2112
    
    #then get deltaR for n
    partvec = ak.zip({
        "pt": truthpt_sel[i][n_mask],
        "eta": trutheta_sel[i][n_mask],
        "phi": truthphi_sel[i][n_mask],
        "mass": truthmass_sel[i][n_mask],
    })
    
    part4Dvec = ak.with_name(partvec, "Momentum4D")
    
    ind = findmin(clusPt_sel[i], clusEta_sel[i], clusPhi_sel[i], clusmass_sel[i], part4Dvec)
    #and for the cluster
    #deltaR = []
    #for j in range(len(clusEta_sel[i])):
     #   print('nclusters ', len(clusEta_sel[i]), i)
      #  clusvec = ak.zip({
       #     "pt": clusPt_sel[i][j],
        #    "eta": clusEta_sel[i][j],
         #   "phi": clusPhi_sel[i][j],
        #  "mass": clusmass_sel[i][j]})
        #clus4Dvec = ak.with_name(clusvec, "Momentum4D")
        
        #deltaR.append(part4Dvec.deltaR(clus4Dvec))
        #print('len deltaR', len(deltaR))
        #print(i,j)
       # if len(deltaR) ==0:
        #    print('exception: empty deltaR') #okay so this does work I just needed to restart the kernel:))
         #   print(part4Dvec, clus4Dvec, part4Dvec.deltaR(clus4Dvec), deltaR, i, j)
        #print(len(deltaR))
        #ind = np.argmin(np.array(ak.flatten(deltaR))) #okay this is better yay!! this is the index of that cluster
    
    mindist_indices.append(ind)

[1.10992777]
[1.10992777 3.81656408]
[0.00656473]
[0.00656473 0.16891152]
[0.30936071]
[0.30936071 0.22919047]
[0.30936071 0.22919047 0.33402571]
[0.30936071 0.22919047 0.33402571 0.2924796 ]
[2.94672513]
[0.25807545]
[0.25807545 0.03192846]
[0.25807545 0.03192846 0.20716299]
[0.25807545 0.03192846 0.20716299 0.2404608 ]
[0.25807545 0.03192846 0.20716299 0.2404608  0.17474909]
[0.25807545 0.03192846 0.20716299 0.2404608  0.17474909 0.02098339]
[0.25807545 0.03192846 0.20716299 0.2404608  0.17474909 0.02098339
 0.28743106]
[0.25807545 0.03192846 0.20716299 0.2404608  0.17474909 0.02098339
 0.28743106 0.23805544]
[0.25807545 0.03192846 0.20716299 0.2404608  0.17474909 0.02098339
 0.28743106 0.23805544 0.24338631]
[0.25807545 0.03192846 0.20716299 0.2404608  0.17474909 0.02098339
 0.28743106 0.23805544 0.24338631 0.23327991]
[0.03318548]
[0.03318548 0.02212914]
[0.03318548 0.02212914 0.02008693]
[0.03318548 0.02212914 0.02008693 4.17345095]
[0.03318548 0.02212914 0.02008693 4.17345095 4.2

ValueError: attempt to get argmin of an empty sequence

In [26]:
np.array(ind)

array([1.3131634], dtype=float32)

In [46]:
deltaR

[<Array [2.84] type='1 * float32'>, <Array [3.94] type='1 * float32'>]

In [15]:
mindist_indices

[(<Array [0] type='1 * int64'>, <Array [0] type='1 * int64'>),
 (<Array [0] type='1 * int64'>, <Array [0] type='1 * int64'>),
 (<Array [1] type='1 * int64'>, <Array [0] type='1 * int64'>),
 (<Array [0] type='1 * int64'>, <Array [0] type='1 * int64'>),
 (<Array [5] type='1 * int64'>, <Array [0] type='1 * int64'>),
 (<Array [2] type='1 * int64'>, <Array [0] type='1 * int64'>),
 (<Array [0] type='1 * int64'>, <Array [0] type='1 * int64'>),
 (<Array [1] type='1 * int64'>, <Array [0] type='1 * int64'>),
 (<Array [2] type='1 * int64'>, <Array [0] type='1 * int64'>),
 (<Array [0] type='1 * int64'>, <Array [0] type='1 * int64'>),
 (<Array [0] type='1 * int64'>, <Array [0] type='1 * int64'>),
 (<Array [0] type='1 * int64'>, <Array [0] type='1 * int64'>),
 (<Array [0] type='1 * int64'>, <Array [0] type='1 * int64'>),
 (<Array [2] type='1 * int64'>, <Array [0] type='1 * int64'>),
 (<Array [0] type='1 * int64'>, <Array [0] type='1 * int64'>),
 (<Array [0] type='1 * int64'>, <Array [0] type='1 * in

In [12]:
partvec = ak.zip({
    "pt": truthpt_sel,
    "eta": trutheta_sel,
    "phi": truthphi_sel,
    "mass": truthmass_sel,
})

clusvec = ak.zip({
    "pt": clusPt_sel,
    "eta": clusEta_sel,
    "phi": clusPhi_sel,
    "mass": clusmass_sel,
})


In [16]:
len(partvec[0]), len(clusvec[0])

(4, 2)

In [14]:
part4Dvec = ak.with_name(partvec, "Momentum4D")
clus4Dvec = ak.with_name(clusvec, "Momentum4D")

In [15]:
deltaR2 = part4Dvec.deltaR(clus4Dvec) #this error is why I needed to make it into a loop

ValueError: in ListOffsetArray64, cannot broadcast nested list

(https://github.com/scikit-hep/awkward-1.0/blob/1.4.0/src/cpu-kernels/awkward_ListArray_broadcast_tooffsets.cpp#L27)